In [ ]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
class net(torch.nn.Module):
    def __init__(self, base_model, base_out_features, num_classes):
        super(net,self).__init__()
        self.base_model=base_model
        self.linear1 = torch.nn.Linear(base_out_features, 512)
        self.output = torch.nn.Linear(512,num_classes)
    def forward(self,x):
        x = F.relu(self.base_model(x))
        x = F.relu(self.linear1(x))
        x = self.output(x)
        return x

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [ ]:
res = torchvision.models.resnet50(pretrained=True)
for param in res.parameters():
    param.requires_grad=False

model = net(base_model=res, base_out_features=res.fc.out_features, num_classes=80)
model = model.to(device)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/졸업논문/best_model.pth', map_location=device))
model.eval()

In [ ]:
!mkdir 'cat_img'

mkdir: cannot create directory ‘cat_img’: File exists


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/졸업논문/data/고양이/cat_train.zip -d /content/cat_img/

In [ ]:
cat_list = os.listdir('./cat_img')
cat_breed=[]

In [ ]:
dir = './cat_img'
cat_img = {'img':cat_list}
cat_img = pd.DataFrame(cat_img)
cat_img.img = cat_img.img.apply(lambda x:dir+'/'+x)

In [ ]:
comp_df = pd.read_csv('./drive/MyDrive/졸업논문/data/labels_수정.csv')
print('Training set: {}'.format(comp_df.shape[0]))

Training set: 6898


In [ ]:
comp_df['label'] = LabelEncoder().fit_transform(comp_df.breed)

dict_df = comp_df[['label','breed']].copy()
dict_df.drop_duplicates(inplace=True)
dict_df.set_index('label',drop=True,inplace=True)
index_to_breed = dict_df.to_dict()['breed']

In [ ]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
class img_dataset(Dataset):
    def __init__(self,dataframe,transform=None,test=False):
        self.dataframe = dataframe
        self.transform = transform
        self.test = test
        
    def __getitem__(self,index):
        x = Image.open(self.dataframe.iloc[index,0]).convert('RGB')
        #x = Image.open(self.dataframe.iloc[index,0])
        #x = cv2.imread(self.dataframe.iloc[index,0])
        
        if self.transform:
            x = self.transform(x)
        if self.test:
            return x
        else:
            y = self.dataframe.iloc[index,1]
            return x,y
        
    def __len__(self):
        return self.dataframe.shape[0]

In [ ]:
train_transformer = transforms.Compose([transforms.RandomResizedCrop(224),
                                        transforms.RandomRotation(15),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

val_transformer = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:

cat_dataset = img_dataset(cat_img,transform=val_transformer,test=True)

In [ ]:
catloader = DataLoader(cat_dataset)

for x in catloader:
  x = x.to(device)
  y = model(x)
  _,predicted = torch.max(y.data, 1)
  predicted = predicted.cpu()
  integer = predicted.numpy()[0]
  cat_breed.append(integer)

print(len(cat_breed))


90079


In [ ]:
cat_index_to_breed = []
for index in cat_breed:
  cat_index_to_breed.append(index_to_breed[index])

In [ ]:
cat_data = {'id':cat_list,'breed':cat_index_to_breed}
cat_data = pd.DataFrame(cat_data)
cat_data.to_csv('./cat_label.csv',index=False)